In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

### Import mamtorch library
You need to have mamtorch folder in this directory and to install mamtorchkernel by launching, from mamtorch folder, "python kernelsetup.py install" (to improve compiling time, install ninja through "pip install ninja")

In [2]:
import mamtorch # Remember to install mamtorchkernel through mamtorch/kernelsetup.py

### Select the GPU
Currently, MAM kernels are implemented only for usage on GPU

In [3]:
# Select GPU
gpu_id = 0
# Check if the GPU is available, and if so, use it
device = torch.device(f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu")
# You need a gpu to use MAM kernel! (No cpu-based implementation available)
if(device == "cpu"):
    raise "No GPU device available! MAM kernels are not available."

### Define a simple feedforward DNN containing a MAM layer

In [4]:
# Define a simple feedforward neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(SimpleNN, self).__init__()
        # Instantiate a MAC fc layer
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        
        # Instantiate a MAM fc layer
        self.fc2_mam = mamtorch.MAMDense(hidden_size1, hidden_size2, bias=True, beta=True, beta_epochs=4)
        self.relu2 = nn.ReLU()
        
        # Instantiate the output layer
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2_mam(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x
    
# Hyperparameters
input_size = 28 * 28  # MNIST image size
hidden_size1 = 512
hidden_size2 = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

Load MNIST dataset and apply transformations

In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Initialize the model, loss function, and optimizer

In [6]:
model = SimpleNN(input_size, hidden_size1, hidden_size2, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train the network

In [ ]:
# Initialize the selection matrix list.
# Here, for each training epoch, we store the number of times each interconnection has been used
# I.E., the selection count
selection_matrix_list = []

print("Validation with random weights...", end="\r")
model.eval() # set evaluation mode

# reset the selection count in the mam layer
model.fc2_mam.reset_selection_count()

correct = 0
total = 0
for images, labels in test_loader:
    images = images.view(-1, 28 * 28).to(device)  # Flatten the input images
    outputs = model(images)
    
    # update the selection count with the results of this batch
    model.fc2_mam.update_selection_count()
    
    # get the predicted class
    _, predicted = torch.max(outputs.data, 1)
    
    # evaluate the correct values agaist the total evaluated
    correct += (predicted == labels.to(device)).sum().item()
    total += labels.size(0)

# evaluate accuracy
accuracy = correct / total
print(f'Validation accuracy on test set: {accuracy*100:.2f}%')

# add evaluated selection count matrix to the list
# (as the sum of the count of interconnections selected as maximum and as minimum)
selection_matrix_list += [model.fc2_mam.max_selection_count + model.fc2_mam.min_selection_count]

# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    start_time = time.perf_counter()
    print(f"Epoch [{epoch + 1}/{num_epochs}]")

    model.train() # se training mode
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28 * 28).to(device)  # Flatten the input images
        outputs = model(images)
        loss = criterion(outputs, labels.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Training step [{i + 1}/{total_step}], Loss: {loss.item():.4f}', end='\r')
    print()
    
    print("Validation...", end="\r")
    model.eval() # set evaluation mode
    
    # reset the selection count in the mam layer
    model.fc2_mam.reset_selection_count()
    
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, 28 * 28).to(device)  # Flatten the input images
        outputs = model(images)
        
        # update the selection count with the results of this batch
        model.fc2_mam.update_selection_count()
        
        # get the predicted class
        _, predicted = torch.max(outputs.data, 1)
        
        # evaluate the correct values agaist the total evaluated
        correct += (predicted == labels.to(device)).sum().item()
        total += labels.size(0)
    
    # evaluate accuracy
    accuracy = correct / total
    print(f'Validation accuracy on test set: {accuracy*100:.2f}%')
    
    # add evaluated selection count matrix to the list
    # (as the sum of the count of interconnections selected as maximum and as minimum)
    selection_matrix_list += [model.fc2_mam.max_selection_count + model.fc2_mam.min_selection_count]
    
    # update the value of beta for vanishing contributes
    model.fc2_mam.adjust_beta(epoch)
    
    print(f"Elapsed time = {time.perf_counter()-start_time:.3f} s")
    
print("Training end.")

Validation accuracy on test set: 7.55%
Epoch [1/10]
Training step [938/938], Loss: 0.2021
Validation accuracy on test set: 84.36%
Elapsed time = 24.372 s
Epoch [2/10]
Training step [938/938], Loss: 0.4547
Validation accuracy on test set: 86.26%
Elapsed time = 24.087 s
Epoch [3/10]
Training step [938/938], Loss: 0.3886
Validation accuracy on test set: 86.44%
Elapsed time = 23.932 s
Epoch [4/10]


### Analysis on the interconnections
1) How many interconnections have been selected at least once for each training epoch?
2) How many times each interconnection has been selected as max/min for each training epoch?

In [7]:
legend_list = []
data_list = []

for i in range(num_epochs+1):
    selected_interconnections = int(torch.sum(selection_matrix_list[i]>0))
    total_interconnections = selection_matrix_list[i].shape[0]*selection_matrix_list[i].shape[1]
    print("Training epoch", i)
    print(f"Number of selected interconnections: {selected_interconnections} "
          f"({selected_interconnections/total_interconnections*100:.2f}%)")
    data_list += [np.histogram(selection_matrix_list[i].cpu().numpy().flatten(), bins=np.max(selection_matrix_list[i].cpu().numpy()))]
    legend_list += [f"epoch {i}"]

NameError: name 'selection_matrix_list' is not defined

In [ ]:
plt.figure(figsize=(8, 5))
cmap = plt.get_cmap('jet')
for i in range(num_epochs):
    plt.plot(data_list[i][1][:-1], data_list[i][0], linewidth=0.5, color=cmap(int(i/num_epochs*255)))
plt.grid()
plt.xlabel("Selection count")
plt.ylabel("Number of interconnections")
plt.yscale("log")
plt.legend(legend_list)
plt.show()

In [9]:
from torch.profiler import profile, record_function, ProfilerActivity

images = test_loader.dataset[batch_size][0].view(-1, 28 * 28).to(device)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, with_stack=True) as prof:
    with record_function("model_inference"):
        outputs = model(images)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

STAGE:2024-01-16 17:08:02 2387956:2387956 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.12%       1.366ms       100.00%        1.096s        1.096s       0.000us         0.00%     220.000us     220.000us          16 b          16 b       8.63 Mb      -8.00 K

STAGE:2024-01-16 17:08:03 2387956:2387956 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2024-01-16 17:08:03 2387956:2387956 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
